In [ ]:

import pandas as pd
import requests, json

date = '20250201'
stock_no = '2317'

html = requests.get('https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=%s&stockNo=%s' % (date,stock_no))
content = json.loads(html.text)
stock_data = content['data']
col_name = content['fields']

df = pd.DataFrame(data=stock_data, columns=col_name)
df.head(20)

KeyError: 'data'

In [16]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta

def fetch_stock_data(stock_no, start_date, end_date):
    all_data = []

    # 逐月抓取數據
    current_date = datetime.strptime(start_date, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y%m%d")

    while current_date <= end_date:
        date_str = current_date.strftime("%Y%m%d")  # 轉換成 YYYYMMDD 格式
        url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={stock_no}'

        response = requests.get(url)
        if response.status_code == 200:
            content = json.loads(response.text)
            if 'data' in content and 'fields' in content:
                stock_data = content['data']
                col_name = content['fields']

                df = pd.DataFrame(data=stock_data, columns=col_name)
                all_data.append(df)

        # 移動到下一個月
        next_month = current_date.month % 12 + 1
        next_year = current_date.year + (1 if current_date.month == 12 else 0)
        current_date = datetime(next_year, next_month, 1)

    # 合併所有 DataFrame
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        return None

# 設定查詢範圍
stock_no = '2454'
start_date = '20250131'
end_date = '20250313'

df = fetch_stock_data(stock_no, start_date, end_date)

if df is not None:
    print(df.head(20))  # 顯示前 20 筆資料
    df.to_csv(f"{stock_no}_{start_date}_{end_date}.csv", index=False)  # 存成 CSV 檔案
else:
    print("無法獲取股票數據")


           日期        成交股數            成交金額       開盤價       最高價       最低價  \
0   114/01/02   7,374,055  10,011,026,495  1,395.00  1,400.00  1,335.00   
1   114/01/03   6,413,343   8,684,840,857  1,365.00  1,370.00  1,335.00   
2   114/01/06   7,760,498  10,980,061,110  1,390.00  1,425.00  1,390.00   
3   114/01/07  13,043,149  19,029,056,158  1,445.00  1,500.00  1,415.00   
4   114/01/08  11,341,648  16,799,397,207  1,500.00  1,500.00  1,460.00   
5   114/01/09   7,283,127  10,619,853,655  1,470.00  1,480.00  1,440.00   
6   114/01/10   7,456,378  10,977,244,936  1,460.00  1,495.00  1,445.00   
7   114/01/13  10,467,556  14,785,600,300  1,435.00  1,460.00  1,385.00   
8   114/01/14   5,672,581   7,934,393,535  1,385.00  1,415.00  1,380.00   
9   114/01/15   6,608,704   9,392,555,305  1,390.00  1,445.00  1,390.00   
10  114/01/16   7,526,306  10,959,335,965  1,450.00  1,470.00  1,430.00   
11  114/01/17   5,593,733   8,016,913,060  1,450.00  1,455.00  1,420.00   
12  114/01/20   4,054,089

使用技術分析

台積電

In [22]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
import pandas_ta as ta  # 技術指標套件

def fetch_stock_data(stock_no, start_date, end_date):
    all_data = []
    current_date = datetime.strptime(start_date, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y%m%d")

    while current_date <= end_date:
        date_str = current_date.strftime("%Y%m%d")
        url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={stock_no}'

        response = requests.get(url)
        if response.status_code == 200:
            content = json.loads(response.text)
            if 'data' in content and 'fields' in content:
                stock_data = content['data']
                col_name = content['fields']

                df = pd.DataFrame(data=stock_data, columns=col_name)
                all_data.append(df)

        next_month = current_date.month % 12 + 1
        next_year = current_date.year + (1 if current_date.month == 12 else 0)
        current_date = datetime(next_year, next_month, 1)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        return None

def process_technical_indicators(df):
    # 轉換民國年為西元年，並將日期轉換成 datetime 格式
    def convert_to_ad_date(roc_date):
        # 將 "113/01/02" 轉換成 "2024/01/02"
        year, month, day = roc_date.split('/')
        year = str(int(year) + 1911)  # 民國轉西元
        return f"{year}/{month}/{day}"
    
    df["日期"] = df["日期"].apply(convert_to_ad_date)
    df["日期"] = pd.to_datetime(df["日期"], format="%Y/%m/%d")
    
    # 轉換數據格式
    df["收盤價"] = df["收盤價"].str.replace(",", "").astype(float)
    df["最高價"] = df["最高價"].str.replace(",", "").astype(float)
    df["最低價"] = df["最低價"].str.replace(",", "").astype(float)
    df["開盤價"] = df["開盤價"].str.replace(",", "").astype(float)

    # 計算技術指標
    df["SMA_5"] = ta.sma(df["收盤價"], length=5)  # 5日均線
    df["SMA_20"] = ta.sma(df["收盤價"], length=20)  # 20日均線
    df["SMA_60"] = ta.sma(df["收盤價"], length=60)  # 60日均線
    df["SMA_50"] = ta.sma(df["收盤價"], length=50)  # 50日均線
    df["SMA_200"] = ta.sma(df["收盤價"], length=200)  # 200日均線

    df["EMA_5"] = ta.ema(df["收盤價"], length=5)  # 5日指數均線
    df["EMA_20"] = ta.ema(df["收盤價"], length=20)  # 20日指數均線

    df["RSI"] = ta.rsi(df["收盤價"], length=14)  # RSI 指標
    
    macd_df = ta.macd(df["收盤價"])
    df["MACD"] = macd_df["MACD_12_26_9"]
    df["MACD_signal"] = macd_df["MACDs_12_26_9"]
    df["MACD_hist"] = macd_df["MACDh_12_26_9"]

    

    return df

def calculate_support_resistance(df):
    """
    根據移動平均線來計算支撐與壓力區間
    - 如果收盤價低於某條均線，則該均線可能成為支撐
    - 如果收盤價高於某條均線，則該均線可能成為壓力
    """
    df = df.copy()

    # 計算支撐位（Support Level）
    df["Support_Level"] = df[["SMA_20", "SMA_50", "SMA_200"]].min(axis=1)

    # 計算壓力位（Resistance Level）
    df["Resistance_Level"] = df[["SMA_20", "SMA_50", "SMA_200"]].max(axis=1)

    return df





# 設定查詢範圍
stock_no = '2330'
start_date = '20240201'
end_date = '20250313'

df = fetch_stock_data(stock_no, start_date, end_date)

if df is not None:
    df = process_technical_indicators(df)  # 計算技術指標
    df = calculate_support_resistance(df)  # 加入移動平均線支撐壓力
    #df = calculate_fibonacci(df)  # 如果有斐波那契回撤計算

    print(df.tail(20))  # 顯示最後 20 筆數據
    df.to_csv(f"{stock_no}_{start_date}_{end_date}_technical.csv", index=False)  # 存成 CSV
else:
    print("無法獲取股票數據")



            日期        成交股數            成交金額     開盤價     最高價     最低價     收盤價  \
243 2025-02-13  35,681,521  38,806,197,999  1090.0  1095.0  1080.0  1090.0   
244 2025-02-14  73,417,323  78,100,130,074  1065.0  1070.0  1060.0  1060.0   
245 2025-02-17  37,215,861  40,057,489,554  1065.0  1085.0  1065.0  1085.0   
246 2025-02-18  24,018,940  26,204,727,000  1085.0  1100.0  1080.0  1100.0   
247 2025-02-19  28,756,849  31,328,641,002  1090.0  1095.0  1085.0  1090.0   
248 2025-02-20  31,108,197  33,537,160,793  1080.0  1085.0  1070.0  1080.0   
249 2025-02-21  31,480,715  34,322,859,039  1085.0  1095.0  1080.0  1095.0   
250 2025-02-24  33,837,606  36,461,265,148  1080.0  1085.0  1075.0  1075.0   
251 2025-02-25  53,174,277  56,084,293,935  1055.0  1060.0  1050.0  1055.0   
252 2025-02-26  42,921,499  45,208,281,264  1045.0  1060.0  1045.0  1060.0   
253 2025-02-27  61,646,900  64,595,715,596  1065.0  1065.0  1040.0  1040.0   
254 2025-03-03  73,279,419  74,076,141,634  1000.0  1020.0  1000

鴻海

In [23]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
import pandas_ta as ta  # 技術指標套件
import numpy as np

def fetch_stock_data(stock_no, start_date, end_date):
    all_data = []
    current_date = datetime.strptime(start_date, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y%m%d")

    while current_date <= end_date:
        date_str = current_date.strftime("%Y%m%d")
        url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={stock_no}'

        response = requests.get(url)
        if response.status_code == 200:
            content = json.loads(response.text)
            if 'data' in content and 'fields' in content:
                stock_data = content['data']
                col_name = content['fields']

                df = pd.DataFrame(data=stock_data, columns=col_name)
                all_data.append(df)

        next_month = current_date.month % 12 + 1
        next_year = current_date.year + (1 if current_date.month == 12 else 0)
        current_date = datetime(next_year, next_month, 1)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        return None

def process_technical_indicators(df):
    def convert_to_ad_date(roc_date):
        year, month, day = roc_date.split('/')
        year = str(int(year) + 1911)  # 民國轉西元
        return f"{year}/{month}/{day}"
    
    df["日期"] = df["日期"].apply(convert_to_ad_date)
    df["日期"] = pd.to_datetime(df["日期"], format="%Y/%m/%d")
    
    # 轉換數據格式並處理 "--" 遺漏值
    numeric_columns = ["收盤價", "最高價", "最低價", "開盤價"]
    for col in numeric_columns:
        df[col] = df[col].replace('--', np.nan)  # 將 "--" 轉換為 NaN
        df[col] = df[col].str.replace(",", "").astype(float)

    # 可選：填補 NaN 或直接刪除
    df.dropna(inplace=True)  # 移除 NaN 行
    # df.fillna(method='ffill', inplace=True)  # 或用前值填補缺失值

    # 計算技術指標
    df["SMA_5"] = ta.sma(df["收盤價"], length=5)  
    df["SMA_20"] = ta.sma(df["收盤價"], length=20)
    df["SMA_60"] = ta.sma(df["收盤價"], length=60)  # 60日均線  
    df["SMA_50"] = ta.sma(df["收盤價"], length=50)  
    df["SMA_200"] = ta.sma(df["收盤價"], length=200)  

    df["EMA_5"] = ta.ema(df["收盤價"], length=5)  
    df["EMA_20"] = ta.ema(df["收盤價"], length=20)  

    df["RSI"] = ta.rsi(df["收盤價"], length=14)  
    
    macd_df = ta.macd(df["收盤價"])
    df["MACD"] = macd_df["MACD_12_26_9"]
    df["MACD_signal"] = macd_df["MACDs_12_26_9"]
    df["MACD_hist"] = macd_df["MACDh_12_26_9"]

    return df

def calculate_support_resistance(df):
    """
    根據移動平均線來計算支撐與壓力區間
    - 如果收盤價低於某條均線，則該均線可能成為支撐
    - 如果收盤價高於某條均線，則該均線可能成為壓力
    """
    df = df.copy()

    # 計算支撐位（Support Level）
    df["Support_Level"] = df[["SMA_20", "SMA_50", "SMA_200"]].min(axis=1)

    # 計算壓力位（Resistance Level）
    df["Resistance_Level"] = df[["SMA_20", "SMA_50", "SMA_200"]].max(axis=1)

    return df





# 設定查詢範圍
stock_no = '2317'
start_date = '20240201'
end_date = '20250313'

df = fetch_stock_data(stock_no, start_date, end_date)

if df is not None:
    df = process_technical_indicators(df)  # 計算技術指標
    df = calculate_support_resistance(df)  # 加入移動平均線支撐壓力
    #df = calculate_fibonacci(df)  # 如果有斐波那契回撤計算

    print(df.tail(20))  # 顯示最後 20 筆數據
    df.to_csv(f"{stock_no}_{start_date}_{end_date}_technical.csv", index=False)  # 存成 CSV
else:
    print("無法獲取股票數據")



            日期        成交股數            成交金額    開盤價    最高價    最低價    收盤價   漲跌價差  \
243 2025-02-13  40,596,733   7,259,602,358  177.0  181.0  177.0  180.0  +3.00   
244 2025-02-14  26,925,038   4,808,438,797  180.0  180.5  177.5  178.0  -2.00   
245 2025-02-17  47,251,327   8,577,655,581  179.5  182.5  179.0  182.5  +4.50   
246 2025-02-18  27,184,244   4,958,070,370  183.0  184.0  181.0  183.5  +1.00   
247 2025-02-19  35,389,548   6,488,063,362  183.5  184.5  182.0  183.0  -0.50   
248 2025-02-20  23,659,372   4,311,924,877  183.0  183.5  181.5  182.0  -1.00   
249 2025-02-21  35,359,340   6,465,535,028  181.5  184.0  181.0  183.0  +1.00   
250 2025-02-24  27,928,907   5,103,161,593  181.0  184.0  180.5  184.0  +1.00   
251 2025-02-25  70,811,787  12,690,044,507  181.5  182.0  177.5  177.5  -6.50   
252 2025-02-26  45,145,771   8,041,430,005  175.0  181.0  174.5  180.5  +3.00   
253 2025-02-27  81,509,723  14,374,897,915  180.0  181.5  174.0  174.0  -6.50   
254 2025-03-03  70,556,492  

聯發科

In [24]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
import pandas_ta as ta  # 技術指標套件
import numpy as np

def fetch_stock_data(stock_no, start_date, end_date):
    all_data = []
    current_date = datetime.strptime(start_date, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y%m%d")

    while current_date <= end_date:
        date_str = current_date.strftime("%Y%m%d")
        url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={stock_no}'

        response = requests.get(url)
        if response.status_code == 200:
            content = json.loads(response.text)
            if 'data' in content and 'fields' in content:
                stock_data = content['data']
                col_name = content['fields']

                df = pd.DataFrame(data=stock_data, columns=col_name)
                all_data.append(df)

        next_month = current_date.month % 12 + 1
        next_year = current_date.year + (1 if current_date.month == 12 else 0)
        current_date = datetime(next_year, next_month, 1)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        return None

def process_technical_indicators(df):
    def convert_to_ad_date(roc_date):
        year, month, day = roc_date.split('/')
        year = str(int(year) + 1911)  # 民國轉西元
        return f"{year}/{month}/{day}"
    
    df["日期"] = df["日期"].apply(convert_to_ad_date)
    df["日期"] = pd.to_datetime(df["日期"], format="%Y/%m/%d")
    
    # 轉換數據格式並處理 "--" 遺漏值
    numeric_columns = ["收盤價", "最高價", "最低價", "開盤價"]
    for col in numeric_columns:
        df[col] = df[col].replace('--', np.nan)  # 將 "--" 轉換為 NaN
        df[col] = df[col].str.replace(",", "").astype(float)

    # 可選：填補 NaN 或直接刪除
    df.dropna(inplace=True)  # 移除 NaN 行
    # df.fillna(method='ffill', inplace=True)  # 或用前值填補缺失值

    # 計算技術指標
    df["SMA_5"] = ta.sma(df["收盤價"], length=5)  
    df["SMA_20"] = ta.sma(df["收盤價"], length=20)
    df["SMA_60"] = ta.sma(df["收盤價"], length=60)  # 60日均線  
    df["SMA_50"] = ta.sma(df["收盤價"], length=50)  
    df["SMA_200"] = ta.sma(df["收盤價"], length=200)  

    df["EMA_5"] = ta.ema(df["收盤價"], length=5)  
    df["EMA_20"] = ta.ema(df["收盤價"], length=20)  

    df["RSI"] = ta.rsi(df["收盤價"], length=14)  
    
    macd_df = ta.macd(df["收盤價"])
    df["MACD"] = macd_df["MACD_12_26_9"]
    df["MACD_signal"] = macd_df["MACDs_12_26_9"]
    df["MACD_hist"] = macd_df["MACDh_12_26_9"]

    return df

def calculate_support_resistance(df):
    """
    根據移動平均線來計算支撐與壓力區間
    - 如果收盤價低於某條均線，則該均線可能成為支撐
    - 如果收盤價高於某條均線，則該均線可能成為壓力
    """
    df = df.copy()

    # 計算支撐位（Support Level）
    df["Support_Level"] = df[["SMA_20", "SMA_50", "SMA_200"]].min(axis=1)

    # 計算壓力位（Resistance Level）
    df["Resistance_Level"] = df[["SMA_20", "SMA_50", "SMA_200"]].max(axis=1)

    return df




# 設定查詢範圍
stock_no = '2454'
start_date = '20240201'
end_date = '20250313'

df = fetch_stock_data(stock_no, start_date, end_date)

if df is not None:
    df = process_technical_indicators(df)  # 計算技術指標
    df = calculate_support_resistance(df)  # 加入移動平均線支撐壓力
    #df = calculate_fibonacci(df)  # 如果有斐波那契回撤計算

    print(df.tail(20))  # 顯示最後 20 筆數據
    df.to_csv(f"{stock_no}_{start_date}_{end_date}_technical.csv", index=False)  # 存成 CSV
else:
    print("無法獲取股票數據")



            日期        成交股數            成交金額     開盤價     最高價     最低價     收盤價  \
243 2025-02-13   4,801,436   7,239,919,170  1495.0  1520.0  1485.0  1520.0   
244 2025-02-14   4,867,021   7,398,870,815  1515.0  1535.0  1505.0  1505.0   
245 2025-02-17   8,095,856  12,386,691,075  1515.0  1545.0  1510.0  1545.0   
246 2025-02-18   5,461,888   8,359,528,915  1550.0  1550.0  1520.0  1535.0   
247 2025-02-19   4,710,329   7,242,322,405  1535.0  1545.0  1525.0  1540.0   
248 2025-02-20   7,150,167  10,837,847,830  1545.0  1545.0  1500.0  1500.0   
249 2025-02-21   5,756,665   8,675,140,570  1500.0  1525.0  1490.0  1510.0   
250 2025-02-24   6,497,488   9,923,578,445  1500.0  1545.0  1495.0  1545.0   
251 2025-02-25   8,502,805  13,096,692,819  1525.0  1560.0  1510.0  1550.0   
252 2025-02-26   8,063,842  12,531,654,210  1555.0  1575.0  1525.0  1560.0   
253 2025-02-27  12,022,234  18,461,918,925  1555.0  1575.0  1515.0  1515.0   
254 2025-03-03  11,544,405  17,065,859,785  1490.0  1495.0  1465